<h3>IS 608 HW 4</h3>
   
Using the dataset available [here](https://github.com/jlaurito/CUNY_IS608/tree/master/lecture4/data) complete the following:
1. Create lists & graphs of the best and worst places to swim in the dataset.
2. The testing of water quality can be sporadic. Which sites have been tested most regularly? Which ones have long gaps between tests? Pick out 5-10 sites and visually compare how regularly their water quality is tested.
3. Is there a relationship between the amount of rain and water quality?  Show this relationship graphically. If you can, estimate the effect of rain on quality at different sites and create a visualization to compare them.

Background: The United States Environmental Protection Agency (EPA) reports Entero counts as colonies (or cells) per 100 ml of water. The federal standard for unacceptable water quality is a single sample value of greater than 110 Enterococcus/100mL, or five or more samples with a geometric mean (a weighted average) greater than 30 Enterococcus/100mL.

In [6]:
# Import modules for analysis and visualization
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.mstats import gmean
import numpy as np
import os.path

In [7]:
# Clean up and import (pandas)
data_url = "https://raw.githubusercontent.com/jlaurito/CUNY_IS608/master/lecture4/data/riverkeeper_data_2013.csv"
raw_data = pd.read_csv(data_url)
# info like what you get with str() in R
raw_data.dtypes 

Site                 object
Date                 object
EnteroCount          object
FourDayRainTotal    float64
SampleCount           int64
dtype: object

In [8]:
# look at the first 25 rows
# remember the first row is the colnames
raw_data.head(25)

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount
0,Hudson above Mohawk River,10/16/2011,1733,1.5,35
1,Hudson above Mohawk River,10/21/2013,4,0.2,35
2,Hudson above Mohawk River,9/21/2013,20,0.0,35
3,Hudson above Mohawk River,8/19/2013,6,0.0,35
4,Hudson above Mohawk River,7/21/2013,31,0.0,35
5,Hudson above Mohawk River,6/4/2013,238,1.2,35
6,Hudson above Mohawk River,10/15/2012,23,1.4,35
7,Hudson above Mohawk River,9/15/2012,11,0.1,35
8,Hudson above Mohawk River,8/18/2012,15,0.3,35
9,Hudson above Mohawk River,7/21/2012,6,0.2,35


In [9]:
# Adjust the data types to make the data easier to work with
raw_data["Site"] = raw_data["Site"].astype('category') # like facets
raw_data["Date"] = pd.to_datetime(raw_data["Date"]) # dates as dates
# We need to remove the greater than and less than symbols and treat "EnteroCount" as an integer -- 'int64'
raw_data["EnteroCount"] = raw_data["EnteroCount"].str.lstrip('><')
raw_data["EnteroCount"] = raw_data["EnteroCount"].astype('int64')

In [10]:
# Check to make sure the changes are correct
raw_data.dtypes 

Site                      category
Date                datetime64[ns]
EnteroCount                  int64
FourDayRainTotal           float64
SampleCount                  int64
dtype: object

In [11]:
raw_data.head(20) # there are no more < or > symbols

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount
0,Hudson above Mohawk River,2011-10-16,1733,1.5,35
1,Hudson above Mohawk River,2013-10-21,4,0.2,35
2,Hudson above Mohawk River,2013-09-21,20,0.0,35
3,Hudson above Mohawk River,2013-08-19,6,0.0,35
4,Hudson above Mohawk River,2013-07-21,31,0.0,35
5,Hudson above Mohawk River,2013-06-04,238,1.2,35
6,Hudson above Mohawk River,2012-10-15,23,1.4,35
7,Hudson above Mohawk River,2012-09-15,11,0.1,35
8,Hudson above Mohawk River,2012-08-18,15,0.3,35
9,Hudson above Mohawk River,2012-07-21,6,0.2,35


1. Create lists & graphs of the best and worst places to swim in the dataset.

In [12]:
# Create a column showing whether or not the water quality is acceptable in a given place
# unnaceptable: 
# 110 Enterococcus/100mL OR
# five or more samples with a geometric mean (a weighted average) > 30 Enterococcus/100mL.

# The prompt is really vague in defining the geometric mean condition - I used gmean()

# When calculated, geometric mean by site of highest 5 samples and single sample >= 110
# shows nearly all sites would have water with unacceptable water quality at some point

raw_data['swim'] = np.where(
    (raw_data.groupby('Site').EnteroCount.transform(max) > 110) 
    | (raw_data.groupby('Site').EnteroCount.transform(lambda group: gmean(group.nlargest(5))) > 30), 
    'unacceptable', 'acceptable')
raw_data.head(10)

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount,swim
0,Hudson above Mohawk River,2011-10-16,1733,1.5,35,unacceptable
1,Hudson above Mohawk River,2013-10-21,4,0.2,35,unacceptable
2,Hudson above Mohawk River,2013-09-21,20,0.0,35,unacceptable
3,Hudson above Mohawk River,2013-08-19,6,0.0,35,unacceptable
4,Hudson above Mohawk River,2013-07-21,31,0.0,35,unacceptable
5,Hudson above Mohawk River,2013-06-04,238,1.2,35,unacceptable
6,Hudson above Mohawk River,2012-10-15,23,1.4,35,unacceptable
7,Hudson above Mohawk River,2012-09-15,11,0.1,35,unacceptable
8,Hudson above Mohawk River,2012-08-18,15,0.3,35,unacceptable
9,Hudson above Mohawk River,2012-07-21,6,0.2,35,unacceptable


In [13]:
# As a result of above criteria being not very useful, I instead grouped the data by site and 
# calculated the mean entero_count for each site

mean_entero = raw_data.groupby(['Site'])['EnteroCount'].mean()
mean_entero.head(15)

Site
125th St. Pier                  179.696970
79th St. mid-channel             47.204082
Albany Rowing Dock              280.944444
Annesville Creek                 83.421053
Athens                          201.314286
Beacon Harbor                    52.657895
Bethlehem Launch Ramp           231.694444
Castle Point, NJ                 37.076923
Castleton                       186.000000
Catskill Creek- East End        261.238095
Catskill Creek- First Bridge    305.947368
Catskill Launch Ramp            132.378378
Cedar Pond Brook                199.702128
Coeymans Landing                186.555556
Cold Spring Harbor               22.542857
Name: EnteroCount, dtype: float64

In [18]:
# Now we can see the top cleanest and least clean (on average) places to swim

# Top ten best places to swim
mean_entero.sort_values()
print "Ten best places to swim"
print "+++++++++++++++++++++++"
mean_entero.head(10)

Ten best places to swim
+++++++++++++++++++++++


Site
Poughkeepsie Drinking Water Intake     8.342105
Croton Point Beach                    15.458333
Stony Point mid-channel               17.340909
Little Stony Point                    17.526316
Poughkeepsie Launch Ramp              17.675676
Haverstraw Bay mid-channel            18.708333
TZ Bridge mid-channel                 21.438596
Cold Spring Harbor                    22.542857
Yonkers mid-channel                   25.019231
Irvington Beach                       28.805556
Name: EnteroCount, dtype: float64

In [19]:
# Ten worst places to swim
print "Ten worst places to swim"
print "++++++++++++++++++++++++"
mean_entero.tail(10)

Ten worst places to swim
++++++++++++++++++++++++


Site
Piermont Pier                               482.165775
Mohawk River at Waterford                   621.057143
Orangetown STP Outfall                      854.192982
Kingsland Pt. Park- Pocantico River         907.857143
Newtown Creek- Dutch Kills                 1205.087719
Upper Sparkill Creek                       1296.072727
Saw Mill River                             1455.760000
Tarrytown Marina                           2205.666667
Newtown Creek- Metropolitan Ave. Bridge    2953.684211
Gowanus Canal                              4206.837838
Name: EnteroCount, dtype: float64

In [20]:
# write the results to a csv and read in for more flexible use (now and later)
mean_entero.to_csv( 'mean_entero.csv' )

In [24]:
mean_entero1 = pd.read_csv('mean_entero.csv')
mean_entero1.columns = ['Site','Average_Entero_Count']
mean_entero1.head()

,Site,Average_Entero_Count
0,Croton Point Beach,15.458333
1,Stony Point mid-channel,17.340909
2,Little Stony Point,17.526316
3,Poughkeepsie Launch Ramp,17.675676
4,Haverstraw Bay mid-channel,18.708333


In [172]:
# Plot the results
%matplotlib inline
sns.set_context("poster")
plt.figure(figsize=(7, 8))
# code from past project as template for seaborn vis
# v_plot = sns.violinplot(x="Search", y="Authors", data=m_results, palette="Blues_d", scale="count")
# v_plot.set_title('Number of Authors per Search Topic')

In [4]:
# 2. The testing of water quality can be sporadic. Which sites have been tested most regularly?
# Which ones have long gaps between tests? 
# Pick out 5-10 sites and visually compare how regularly their water quality is tested.

In [5]:
# 3. Is there a relationship between the amount of rain and water quality? 
# Show this relationship graphically. 
# If you can, estimate the effect of rain on quality at different sites and create a visualization to compare them.

In [25]:
# Determine the most recent reading for each site
# maxDate = raw_data.groupby(by=["Site"])["Date"].max()
# maxDate = dBySiteMaxDate.reset_index()
# maxDate